In [1]:
# Paso 1: Instalar Dependencias
%pip install backtesting pandas numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.8 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 2.6 MB/s eta 0:00:00
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173916 sha256=a3ee44b36d7416a6c38a2ca06f8f42ff971413c38dacc4f8d56889053d113f70
  Stored in directory: /home/codespace/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Paso 2: Importar las Librerías Necesarias
import pandas as pd
import numpy as np
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG

In [10]:
# Paso 3: Definir la Estrategia de Mean Reversion
class MeanReversionStrategy(Strategy):
    def init(self):
        # Inicializa las Bandas de Bollinger
        price = self.data.Close
        self.ma = self.I(SMA, price, 20)  # Media Móvil Simple de 20 períodos
        self.upper_band = self.ma + 2 * price.rolling(20).std()
        self.lower_band = self.ma - 2 * price.rolling(20).std()

    def next(self):
        price = self.data.Close[-1]

        # Señal de compra: el precio cruza por debajo de la banda inferior
        if crossover(self.lower_band, price):
            self.buy()

        # Señal de venta: el precio cruza por encima de la banda superior
        elif crossover(price, self.upper_band):
            self.sell()


In [17]:
# importar data 

datapath = '../backtesting/data/BTCUSDT-5m-2023-06-01.csv'
df = pd.read_csv(datapath, parse_dates=True, index_col=0, )
# data = (df / 1e6).assign(Volume=df.Volume * 1e6)  # μBTC OHLC prices
df.head()

/tmp/ipykernel_44474/992816730.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(datapath, parse_dates=True, index_col=0, )


,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,2023-02-01 00:00:00,23119.5,23161.4,23117.4,23126.8,1824.814,2023-02-01 00:04:59.999,4.222120e+07,12293,1019.472,2.358897e+07,0
1,2023-02-01 00:05:00,23126.8,23129.0,23051.3,23063.2,3383.690,2023-02-01 00:09:59.999,7.810423e+07,20164,1306.681,3.015745e+07,0
2,2023-02-01 00:10:00,23063.2,23089.2,23027.8,23084.6,2656.528,2023-02-01 00:14:59.999,6.125506e+07,15555,1199.793,2.766924e+07,0
3,2023-02-01 00:15:00,23084.6,23087.6,23061.1,23067.0,781.141,2023-02-01 00:19:59.999,1.802459e+07,7080,348.442,8.040091e+06,0
4,2023-02-01 00:20:00,23067.0,23083.8,23061.6,23079.9,678.064,2023-02-01 00:24:59.999,1.564601e+07,6295,357.605,8.251490e+06,0


In [16]:
df = pd.DataFrame(GOOG)
df.rename(columns={"Open": "open", "High": "high", "Low": "low", "Close": "close", "Volume": "volume"}, inplace=True)
df.head()

,open,high,low,close,volume
2004-08-19,100.00,104.06,95.96,100.34,22351900
2004-08-20,101.01,109.08,100.50,108.31,11428600
2004-08-23,110.75,113.48,109.05,109.40,9137200
2004-08-24,111.24,111.60,103.57,104.87,7631300
2004-08-25,104.96,108.00,103.88,106.00,4598900


In [13]:
bt = Backtest(df, MeanReversionStrategy, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot()

/home/codespace/.python/current/lib/python3.10/site-packages/backtesting/backtesting.py:1054: FutureWarning: Index.is_numeric is deprecated. Use pandas.api.types.is_any_real_numeric_dtype instead
  (data.index.is_numeric() and
/tmp/ipykernel_44474/591848084.py:1: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, MeanReversionStrategy, cash=10000, commission=.002)
/tmp/ipykernel_44474/591848084.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MeanReversionStrategy, cash=10000, commission=.002)


AttributeError: '_Array' object has no attribute 'rolling'